In [12]:
import pandas as pd
import seaborn as sns
import numpy as np

#initial data import
df = pd.read_excel('Data.xlsx')
#want to see all columns
pd.set_option('display.max_columns', None)

### Data Exploration/Tidying

To do:

- ~~Remove individual salary columns~~
- ~~Move total salary between term and employment~~
- Look into employment status columns
- ~~Split up loan purpose column into each type~~
- ~~Add multiple customers indicator column~~
- ~~Application type and industry sector need to be encoded~~
- ~~Can we use age or not?~~
- ~~Can we use nationality? most are irish obviously~~
- ~~Make existing mortgate to salary mandated binary~~
- ~~Make LT 1 year values more useable~~
- ~~Work out what status means~~

In [13]:
df

,Application Date (YYYYMM),Term,Customer1 Salary €K,Customer2 Salary €K,Customer1 Employment Status,Customer2 Emp Status,Loan Purpose,Application Type,Industry Sector,Customer1 Age (YYMM),Customer2 Age (YYMM),National of …,CA Months of Unauthorised Debit Interest L6M,CA Average Number of Transactions L6M,CA Average Balance L6M,CA Average Credit Turnover L6M,Existing Mortgage,Existing Current Account,Existing Loan,Existing Savings Acc,Salary Mandated to Same Bank Current Account,Time with Bank Years,Decision,Status,Status - G/B/R
0,201001,27,40,0,F,NaN,REF/RES/TAX,S,PS,2908,0,IRE,0,24,317,3196,N,Y,Y,Y,Y,10,Declined,NaN,R
1,201001,18,47,0,T,NaN,HOR,S,RS,5905,0,IRE,0,22,6021,7090,N,Y,NaN,N,Y,14,Declined,NaN,R
2,201001,60,40,0,F,NaN,REF/RES/TAX,S,E,2704,0,IRE,4,24,1357,1922,Y,Y,Y,N,N,6,Declined,NaN,R
3,201001,12,39,0,F,NaN,EDU,S,FS,2701,0,IRE,1,39,505,3995,N,Y,Y,N,Y,3,Declined,NaN,R
4,201001,60,15,0,F,NaN,REF/RES/TAX,S,RS,5901,0,IRE,1,14,409,1318,N,Y,Y,N,Y,13,Declined,NaN,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12813,201012,24,29,0,F,NaN,HFU,S,M,4600,0,IRE,3,54,994,6247,Y,Y,NaN,N,Y,8,Declined,NaN,R
12814,201012,60,9,32,S,U,REF/RES/TAX,J,O,5300,5304,IRE,0,63,2993,3117,N,Y,Y,N,N,9,Declined,NaN,R
12815,201012,42,33,0,F,NaN,VHU,S,PS,5011,0,IRE,0,12,122,1988,N,Y,NaN,N,N,LT 1 year,Declined,NaN,R
12816,201012,60,18,29,S,F,REF/RES/TAX,J,C,4002,3811,IRE,5,60,5705,5243,Y,Y,NaN,N,N,8,Approved,1,G


In [14]:
#combine salaries of partners
df["Total Salary"] = df.iloc[:,2]+df.iloc[:,3]
df

,Application Date (YYYYMM),Term,Customer1 Salary €K,Customer2 Salary €K,Customer1 Employment Status,Customer2 Emp Status,Loan Purpose,Application Type,Industry Sector,Customer1 Age (YYMM),Customer2 Age (YYMM),National of …,CA Months of Unauthorised Debit Interest L6M,CA Average Number of Transactions L6M,CA Average Balance L6M,CA Average Credit Turnover L6M,Existing Mortgage,Existing Current Account,Existing Loan,Existing Savings Acc,Salary Mandated to Same Bank Current Account,Time with Bank Years,Decision,Status,Status - G/B/R,Total Salary
0,201001,27,40,0,F,NaN,REF/RES/TAX,S,PS,2908,0,IRE,0,24,317,3196,N,Y,Y,Y,Y,10,Declined,NaN,R,40
1,201001,18,47,0,T,NaN,HOR,S,RS,5905,0,IRE,0,22,6021,7090,N,Y,NaN,N,Y,14,Declined,NaN,R,47
2,201001,60,40,0,F,NaN,REF/RES/TAX,S,E,2704,0,IRE,4,24,1357,1922,Y,Y,Y,N,N,6,Declined,NaN,R,40
3,201001,12,39,0,F,NaN,EDU,S,FS,2701,0,IRE,1,39,505,3995,N,Y,Y,N,Y,3,Declined,NaN,R,39
4,201001,60,15,0,F,NaN,REF/RES/TAX,S,RS,5901,0,IRE,1,14,409,1318,N,Y,Y,N,Y,13,Declined,NaN,R,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12813,201012,24,29,0,F,NaN,HFU,S,M,4600,0,IRE,3,54,994,6247,Y,Y,NaN,N,Y,8,Declined,NaN,R,29
12814,201012,60,9,32,S,U,REF/RES/TAX,J,O,5300,5304,IRE,0,63,2993,3117,N,Y,Y,N,N,9,Declined,NaN,R,41
12815,201012,42,33,0,F,NaN,VHU,S,PS,5011,0,IRE,0,12,122,1988,N,Y,NaN,N,N,LT 1 year,Declined,NaN,R,33
12816,201012,60,18,29,S,F,REF/RES/TAX,J,C,4002,3811,IRE,5,60,5705,5243,Y,Y,NaN,N,N,8,Approved,1,G,47


In [15]:
#count number of rejects - about half and half (should be fine for not bothering with inference)
#must address the bias issue!!!
(df["Status - G/B/R"]=="R").sum()

6344

In [16]:
#drop separated customer salary columns
df = df.drop(df.columns[2:4],axis=1)

In [17]:
#sanity check, make sure most purposes is 3 (3x3 letters + 2 slash)
max(df["Loan Purpose"].str.len())

#split up each of three purposes into their own column
new = df["Loan Purpose"].str.split("/", n = 2, expand = True)

#replace none with nan
new = new.fillna(value=np.nan)

#add new columns into our data frame corresponding to each distinct purpose
df["Purpose1"] = new[0]
df["Purpose2"] = new[1]
df["Purpose3"] = new[2]

#remove loan purpose column
df = df.drop(["Loan Purpose"],axis=1)

In [18]:
#add new variable for if there are multiple customers (i.e. married couple etc)
df['Multiple Customers'] = np.where(df['Customer2 Age (YYMM)'] == 0 , 0, 1)

In [19]:
df['Single Purpose'] = df["Purpose2"].isnull()*1

In [20]:
#check for any other non-numeric values
df["Time with Bank Years"].unique()

#also need to deal with New as well as LT 1 year

#have a look at 99 years (could be error)

array([10, 14, 6, 3, 13, 15, 11, 8, 17, 4, 9, 2, 5, 16, 20, 'New', 18, 12,
       1, 7, 19, 'LT 1 year', 22, 21, 32, 24, 99, 25, 23, 33, 26, 31, 29,
       27, 28, 30], dtype=object)

In [21]:
#indices of customers with 99 years at bank
error_idx = np.where(df["Time with Bank Years"]==99)

In [22]:
#example of column w 99 years at bank - clearly error so just remove these entries
df.iloc[1097,:]

Application  Date (YYYYMM)                        201002
Term                                                  60
Customer1 Employment Status                            F
Customer2 Emp Status                                 NaN
Application Type                                       S
Industry Sector                                       RS
Customer1 Age (YYMM)                                4405
Customer2 Age (YYMM)                                   0
National of …                                        IRE
CA Months of Unauthorised Debit Interest L6M           1
CA Average Number of Transactions L6M                 10
CA Average Balance L6M                               757
CA Average Credit Turnover L6M                      2510
Existing Mortgage                                      Y
Existing Current Account                               Y
Existing Loan                                        NaN
Existing Savings Acc                                   Y
Salary Mandated to Same Bank Cu

In [23]:
#remove 99 year rows
df = df[df["Time with Bank Years"] != 99]

In [24]:
#replace LT year with 0 years
df.loc[df["Time with Bank Years"]=="LT 1 year","Time with Bank Years"]=0
#replace New with 0 years
df.loc[df["Time with Bank Years"]=="New","Time with Bank Years"]=0
#check all value now numeric
df["Time with Bank Years"].unique()

array([10, 14, 6, 3, 13, 15, 11, 8, 17, 4, 9, 2, 5, 16, 20, 0, 18, 12, 1,
       7, 19, 22, 21, 32, 24, 25, 23, 33, 26, 31, 29, 27, 28, 30],
      dtype=object)

In [25]:
#delete status column - not needed
df = df.drop("Status",axis=1)

In [26]:
nationality_column = df.columns.values.tolist()[8]

#delete nationality column to avoid discrimination
df = df.drop(nationality_column,axis=1)

In [27]:
#remove date column, not necessary
df = df.iloc[:,1:]

In [28]:
#only use accepted applications
df = df[df["Status - G/B/R"] != "R"]

In [29]:
df = pd.DataFrame(df)
df.dtypes

Term                                             int64
Customer1 Employment Status                     object
Customer2 Emp Status                            object
Application Type                                object
Industry Sector                                 object
Customer1 Age (YYMM)                             int64
Customer2 Age (YYMM)                             int64
CA Months of Unauthorised Debit Interest L6M     int64
CA Average Number of Transactions L6M            int64
CA Average Balance L6M                           int64
CA Average Credit Turnover L6M                   int64
Existing Mortgage                               object
Existing Current Account                        object
Existing Loan                                   object
Existing Savings Acc                            object
Salary Mandated to Same Bank Current Account    object
Time with Bank Years                            object
Decision                                        object
Status - G

In [30]:
#coerce time with bank to be numeric
df["Time with Bank Years"] = pd.to_numeric(df["Time with Bank Years"])

In [31]:
df.dtypes

Term                                             int64
Customer1 Employment Status                     object
Customer2 Emp Status                            object
Application Type                                object
Industry Sector                                 object
Customer1 Age (YYMM)                             int64
Customer2 Age (YYMM)                             int64
CA Months of Unauthorised Debit Interest L6M     int64
CA Average Number of Transactions L6M            int64
CA Average Balance L6M                           int64
CA Average Credit Turnover L6M                   int64
Existing Mortgage                               object
Existing Current Account                        object
Existing Loan                                   object
Existing Savings Acc                            object
Salary Mandated to Same Bank Current Account    object
Time with Bank Years                             int64
Decision                                        object
Status - G

In [32]:
#irrelevant column dropped as we know all the cases we want are accepted
df = df.drop("Decision",axis = 1)

In [33]:
df = df.rename(columns={"Customer1 Employment Status": "C1 Emp", "Customer2 Emp Status": "C2 Emp", "Application Type": 
                  "Type", "Industry Sector": "Sector", "Customer1 Age (YYMM)": "Age1", "Customer2 Age (YYMM)": 
                   "Age2", "CA Months of Unauthorised Debit Interest L6M": "MUDI", "CA Average Number of Transactions L6M":
                   "Avg Transactions", "CA Average Balance L6M": "Avg Bal", "CA Average Credit Turnover L6M":
                   "Avg Turnover","Salary Mandated to Same Bank Current Account":"SalaryAcc"})

In [35]:
df.groupby(["Status - G/B/R"]).mean()

,Term,Age1,Age2,MUDI,Avg Transactions,Avg Bal,Avg Turnover,Time with Bank Years,Total Salary,Multiple Customers,Single Purpose
Status - G/B/R,,,,,,,,,,,
B,51.715152,4197.066667,1107.848485,2.816162,22.523232,2426.793939,3048.048485,8.858586,33.527273,0.262626,0.208081
G,40.662033,4195.688159,972.489198,1.237649,29.866689,2244.754982,3885.128789,9.713281,43.778932,0.231117,0.717970


In [36]:
df.groupby(["Existing Mortgage","Status - G/B/R"]).size()

Existing Mortgage  Status - G/B/R
N                  B                  332
                   G                 3445
Y                  B                  163
                   G                 2526
dtype: int64

In [37]:
df.groupby(["Type","Status - G/B/R"]).size()

Type  Status - G/B/R
J     B                  125
      G                 1349
M     B                    5
      G                   31
S     B                  365
      G                 4591
dtype: int64

In [38]:
df.groupby(["C1 Emp","Status - G/B/R"]).size()

C1 Emp  Status - G/B/R
C       B                    1
        G                   11
F       B                  252
        G                 4613
H       B                    9
        G                   62
I       G                    3
M       B                    1
        G                   15
P       B                   34
        G                  308
R       B                   14
        G                  219
S       B                   83
        G                  447
T       B                    1
        G                   39
U       B                   80
        G                  168
X       B                    4
        G                   23
Z       G                    2
dtype: int64

In [39]:
df.groupby(["Single Purpose","Status - G/B/R"]).size()

Single Purpose  Status - G/B/R
0               B                  392
                G                 1684
1               B                  103
                G                 4287
dtype: int64

In [40]:
#taking the numeric columns only and dropping age of secondary customer
df_num = df.select_dtypes(include=np.number)
df_num["Outcome"] = df["Status - G/B/R"]
df_num = df_num.drop("Age2",axis=1)
df_num

<ipython-input-40-01df8aedd9ad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_num["Outcome"] = df["Status - G/B/R"]


,Term,Age1,MUDI,Avg Transactions,Avg Bal,Avg Turnover,Time with Bank Years,Total Salary,Multiple Customers,Single Purpose,Outcome
9,60,5606,5,35,202,5400,13,43,0,0,G
10,60,6504,4,30,576,2932,11,22,0,1,G
11,60,4909,0,54,1409,7122,8,78,1,1,G
14,60,5608,6,1,4642,100,10,45,0,0,B
15,60,3601,0,33,1233,3597,3,57,0,1,G
...,...,...,...,...,...,...,...,...,...,...,...
12798,60,2907,4,34,599,3203,3,44,0,1,G
12804,12,4907,0,22,49,2437,4,32,0,1,G
12807,60,2800,1,23,2465,5934,9,52,0,1,G
12808,12,5105,5,3,120,485,7,31,0,1,G


In [41]:
df_num.to_csv("dfnum.csv")